In [1]:
import pandas as pd
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/26 10:02:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark.version

'3.5.4'

In [5]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet

--2025-02-26 10:05:37--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 3.160.226.111, 3.160.226.161, 3.160.226.228, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.160.226.111|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64346071 (61M) [binary/octet-stream]
Saving to: ‘yellow_tripdata_2024-10.parquet’

yellow_tripdata_202 100%[===================>]  61.36M   301MB/s    in 0.2s    

2025-02-26 10:05:37 (301 MB/s) - ‘yellow_tripdata_2024-10.parquet’ saved [64346071/64346071]



In [9]:
df = spark.read.parquet("yellow_tripdata_2024-10.parquet").repartition(4)
df.write.mode("overwrite").parquet("homework2025/")
!du -sh homework2025/*

0	homework2025/_SUCCESS
23M	homework2025/part-00000-12af7407-9f42-4d4d-9a9a-7052f7b6655c-c000.snappy.parquet
23M	homework2025/part-00001-12af7407-9f42-4d4d-9a9a-7052f7b6655c-c000.snappy.parquet
23M	homework2025/part-00002-12af7407-9f42-4d4d-9a9a-7052f7b6655c-c000.snappy.parquet
23M	homework2025/part-00003-12af7407-9f42-4d4d-9a9a-7052f7b6655c-c000.snappy.parquet


In [21]:
df.createOrReplaceTempView('homework2025')

In [29]:
trip_count = spark.sql("""
    SELECT COUNT(*) AS trip_count
    FROM homework2025
    WHERE tpep_pickup_datetime BETWEEN '2024-10-15 00:00:00' AND '2024-10-15 23:59:59'
""")

trip_count.show()

+----------+
|trip_count|
+----------+
|    128893|
+----------+



In [31]:
spark.sql("""
SELECT tpep_pickup_datetime
,tpep_dropoff_datetime
,(unix_timestamp(tpep_dropoff_datetime) - unix_timestamp(tpep_pickup_datetime)) /3600 as hours
FROM homework2025
ORDER BY hours desc
""").show()

+--------------------+---------------------+------------------+
|tpep_pickup_datetime|tpep_dropoff_datetime|             hours|
+--------------------+---------------------+------------------+
| 2024-10-16 13:03:49|  2024-10-23 07:40:53|162.61777777777777|
| 2024-10-03 18:47:25|  2024-10-09 18:06:55|           143.325|
| 2024-10-22 16:00:55|  2024-10-28 09:46:33|137.76055555555556|
| 2024-10-18 09:53:32|  2024-10-23 04:43:37|114.83472222222223|
| 2024-10-21 00:36:24|  2024-10-24 18:30:18| 89.89833333333333|
| 2024-10-20 13:30:52|  2024-10-24 06:57:38| 89.44611111111111|
| 2024-10-22 16:04:52|  2024-10-25 14:22:49| 70.29916666666666|
| 2024-10-12 19:32:51|  2024-10-15 15:07:15| 67.57333333333334|
| 2024-10-17 17:58:18|  2024-10-20 12:02:18| 66.06666666666666|
| 2024-10-21 14:28:21|  2024-10-23 12:53:42|           46.4225|
| 2024-10-20 18:58:28|  2024-10-22 13:17:00| 42.30888888888889|
| 2024-10-24 20:29:58|  2024-10-26 10:58:25| 38.47416666666667|
| 2024-10-23 23:52:02|  2024-10-25 09:49

In [33]:
!wget wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

--2025-02-26 10:36:22--  http://wget/
Resolving wget (wget)... failed: Temporary failure in name resolution.
wget: unable to resolve host address ‘wget’
--2025-02-26 10:36:22--  https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 3.160.226.85, 3.160.226.228, 3.160.226.161, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.160.226.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12331 (12K) [text/csv]
Saving to: ‘taxi_zone_lookup.csv’

taxi_zone_lookup.cs 100%[===================>]  12.04K  --.-KB/s    in 0s      

2025-02-26 10:36:22 (110 MB/s) - ‘taxi_zone_lookup.csv’ saved [12331/12331]

FINISHED --2025-02-26 10:36:22--
Total wall clock time: 0.05s
Downloaded: 1 files, 12K in 0s (110 MB/s)


In [42]:
lookup_spark_df = spark.read.option("header", "true").csv("taxi_zone_lookup.csv")
lookup_spark_df.createOrReplaceTempView('lookup')

In [45]:
spark.sql("""
SELECT count(*) as cnt
,PUlocationID
,Borough
,Zone
FROM homework2025 inner join lookup
on PUlocationID = LocationID
GROUP BY PUlocationID, Borough, Zone
ORDER BY cnt asc
""").show()

+---+------------+-------------+--------------------+
|cnt|PUlocationID|      Borough|                Zone|
+---+------------+-------------+--------------------+
|  1|         105|    Manhattan|Governor's Island...|
|  2|           5|Staten Island|       Arden Heights|
|  2|         199|        Bronx|       Rikers Island|
|  3|           2|       Queens|         Jamaica Bay|
|  3|         111|     Brooklyn| Green-Wood Cemetery|
|  4|         204|Staten Island|   Rossville/Woodrow|
|  4|          44|Staten Island|Charleston/Totten...|
|  4|          84|Staten Island|Eltingville/Annad...|
|  4|         245|Staten Island|       West Brighton|
|  4|         187|Staten Island|       Port Richmond|
|  6|          59|        Bronx|        Crotona Park|
|  6|         109|Staten Island|         Great Kills|
|  7|         118|Staten Island|Heartland Village...|
|  7|         156|Staten Island|     Mariners Harbor|
|  9|         176|Staten Island|             Oakwood|
|  9|         206|Staten Isl